In [1]:
%reload_ext autoreload
%autoreload 2

import json
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import statsmodels.api as sm
from lib import utils


mpl.rcParams['font.family'] = 'times new roman'
mpl.rcParams['font.size'] = 16

In [2]:
df = pd.read_csv('valid_sequence_data.csv',index_col=0)
df['total_money_heuristic'] = df['value_surplus'] == 60
(df.groupby('worker_id')['total_money_heuristic'].sum() >= 12).sum()

62

In [3]:
with open('bootstrap_rlm_fe_label_k.pkl', 'rb') as file:
    bootstrap_rlm_fe_label_k = pickle.load(file)

print(bootstrap_rlm_fe_label_k.conf_int())
print('M-W Criterion:',bootstrap_rlm_fe_label_k.muller_welsh_criterion())

                    median_coef        se       mad   ci_lower   ci_upper
const                 52.030609  0.619995  0.381180  50.921611  53.337420
front_amount_6m_0     -0.016674  0.002263  0.001520  -0.021366  -0.012207
front_amount_12m_0    -0.021383  0.002305  0.001625  -0.025773  -0.017041
front_amount_6m_1     -0.000709  0.000584  0.000390  -0.001880   0.000391
front_amount_12m_1    -0.000920  0.000589  0.000416  -0.001986   0.000200
choice_peli            2.452066  0.658568  0.406990   1.091811   3.717946
M-W Criterion: 119.0131305639573


In [4]:
with open('bootstrap_rlm_fe_baseline_k.pkl', 'rb') as file:
    bootstrap_rlm_fe_baseline_k = pickle.load(file)

print(bootstrap_rlm_fe_baseline_k.conf_int())
print('M-W Criterion:',bootstrap_rlm_fe_baseline_k.muller_welsh_criterion())

                  median_coef        se       mad   ci_lower   ci_upper
const               51.661528  0.628230  0.364891  50.626012  53.107281
front_amount_6m     -0.004375  0.000757  0.000503  -0.005894  -0.002865
front_amount_12m    -0.005661  0.000801  0.000542  -0.007276  -0.004198
choice_peli          2.298416  0.704236  0.427666   0.852563   3.533185
M-W Criterion: 125.06913336116959


In [5]:
with open('bootstrap_rlm_pool_label_k.pkl', 'rb') as file:
    bootstrap_rlm_pool_label_k = pickle.load(file)

print(bootstrap_rlm_pool_label_k.conf_int())
print('M-W Criterion:',bootstrap_rlm_pool_label_k.muller_welsh_criterion())

                    median_coef        se       mad   ci_lower   ci_upper
const                 57.556411  0.598688  0.385843  56.236788  58.677463
front_amount_6m_0     -0.065331  0.003367  0.002189  -0.072133  -0.058980
front_amount_12m_0    -0.068226  0.003271  0.002219  -0.074896  -0.061924
front_amount_6m_1      0.009486  0.001360  0.000925   0.007023   0.012269
front_amount_12m_1     0.008855  0.001456  0.000951   0.006064   0.011924
choice_peli           -1.396088  0.530126  0.348073  -2.344830  -0.311673
M-W Criterion: 214.36420307253906


In [6]:
with open('bootstrap_rlm_pool_baseline_k.pkl', 'rb') as file:
    bootstrap_rlm_pool_baseline_k = pickle.load(file)

print(bootstrap_rlm_pool_baseline_k.conf_int())
print('M-W Criterion:',bootstrap_rlm_pool_baseline_k.muller_welsh_criterion())

                  median_coef        se       mad   ci_lower   ci_upper
const               57.124607  0.883012  0.599287  55.342657  58.765153
front_amount_6m     -0.002598  0.002071  0.001404  -0.006911   0.001501
front_amount_12m    -0.003847  0.002160  0.001368  -0.008345   0.000186
choice_peli         -2.247314  0.758774  0.496706  -3.720454  -0.732289
M-W Criterion: 412.8278249198197


In [8]:
with open('model_reg_result.json', 'r') as f:
    reg_result = json.load(f)

reg_result

{'ols_pool_baseline': {'aic': 19537.883811867527,
  'bic': 19560.643126542473,
  'rsquared_adj': 0.0003826463527634161,
  'nobs': 2186.0,
  'coef_name': ['const', 'front_amount_6m', 'front_amount_12m', 'choice_peli'],
  'params': [53.7422718190243,
   -0.005311989785595047,
   -0.0056402855358125194,
   -0.23853451678695498],
  'tvalues': [44.3075440055407,
   -1.6761094597100399,
   -1.7752756159526175,
   -0.21983465225376847],
  'pvalues': [2.2268088163683902e-306,
   0.09386001389019372,
   0.07599175534760841,
   0.8260204946794963],
  'ci_lower': [51.363639013498826,
   -0.011527029593019334,
   -0.011870806520258688,
   -2.3663998622226616],
  'ci_upper': [56.12090462454978,
   0.0009030500218292405,
   0.000590235448633649,
   1.8893308286487516]},
 'ols_pool_label': {'aic': 19165.413669252583,
  'bic': 19199.552641265003,
  'rsquared_adj': 0.15775508864203125,
  'nobs': 2186.0,
  'coef_name': ['const', 'choice_peli'],
  'params': [52.442832165518055, 1.4188576449295167],
  'tv

In [27]:
bootstrap_rlm_fe_baseline_k.model.bse

const               0.361266
front_amount_6m     0.000988
front_amount_12m    0.000988
choice_peli         0.318851
worker_id_434923    1.752982
                      ...   
worker_id_435123    1.752982
worker_id_435124    1.734972
worker_id_435130    1.752982
worker_id_435131    1.752982
worker_id_435132    1.752982
Length: 161, dtype: float64

In [35]:
bootstrap_rlm_fe_baseline_k.subject_cluster_se()

array([47.63746883,  0.12465448,  0.12465448, 42.56953776])

In [32]:
bootstrap_rlm_fe_baseline_k.subject_cluster_pvalues()

array([0.2767133 , 0.96949225, 0.95898034, 0.95913776])

In [33]:
bootstrap_rlm_fe_baseline_k.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                            Robust linear Model Regression Results                                            
==============================================================================================================================
Dep. Variable:                                                          value_surplus   No. Observations:                 2198
Model:                                                                            RLM   Df Residuals:                     2039
Method:                                                                          IRLS   Df Model:                          158
Norm:                                                                          HuberT                                         
Scale Est.:        <statsmodels.robust.scale.HuberScale object at 0x0000022493C40370>                                         
Cov Type:                                                                          H1                                         
Date:                                                                Sun, 04 Feb 2024                                         
Time:                                                                        02:47:45                                         
No. Iterations:                                                                    42                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               51.8297      0.361    143.467      0.000      51.122      52.538
front_amount_6m     -0.0048      0.001     -4.824      0.000      -0.007      -0.003
front_amount_12m    -0.0064      0.001     -6.488      0.000      -0.008      -0.004
choice_peli          2.1813      0.319      6.841      0.000       1.556       2.806
worker_id_434923     7.8275      1.753      4.465      0.000       4.392      11.263
worker_id_434926     7.4703      1.753      4.261      0.000       4.035      10.906
worker_id_434927    12.4703      1.753      7.114      0.000       9.035      15.906
worker_id_434928   -23.9011      1.735    -13.776      0.000     -27.302     -20.501
worker_id_434930     7.4703      1.753      4.261      0.000       4.035      10.906
worker_id_434931     7.4703      1.753      4.261      0.000       4.035      10.906
worker_id_434932    53.0491      1.753     30.262      0.000      49.613      56.485
worker_id_434934    -0.9583      1.753     -0.547      0.585      -4.394       2.478
worker_id_434935    -7.7217      1.753     -4.405      0.000     -11.157      -4.286
worker_id_434936   -23.3630      1.753    -13.328      0.000     -26.799     -19.927
worker_id_434937   -52.1783      1.735    -30.074      0.000     -55.579     -48.778
worker_id_434940     7.4703      1.753      4.261      0.000       4.035      10.906
worker_id_434941    10.3659      1.735      5.975      0.000       6.965      13.766
worker_id_434942   -10.2164      1.753     -5.828      0.000     -13.652      -6.781
worker_id_434943    18.1018      1.753     10.326      0.000      14.666      21.538
worker_id_434944     8.8570      1.735      5.105      0.000       5.457      12.257
worker_id_434945     7.4703      1.753      4.261      0.000       4.035      10.906
worker_id_434946   -14.0999      1.753     -8.043      0.000     -17.536     -10.664
worker_id_434947     9.6516      1.735      5.563      0.000       6.251      13.052
worker_id_434949   -14.3154      1.753     -8.166      0.000     -17.751     -10.880
worker_id_434952    10.4522      1.735      6.024      0.000       7.052      13.853
worker_id_434953     7.0837      1.753      4.041      0.000       3.648      10.519
worker_id_434954     7.4703      1.753      4.261      0.000       4.035      10.906
worke